# LARUN TinyML - Kaggle Training Notebook

**Train the LARUN exoplanet detection model using Kaggle's FREE GPU**

Created by: Padmanaban Veeraragavalu (Larun Engineering)

---

## Kaggle Setup:
1. Click **Settings** (right sidebar) → **Accelerator** → **GPU T4 x2**
2. Click **Run All** or run cells one by one
3. Output files saved to `/kaggle/working/` (persists!)

**Kaggle Benefits:**
- 30 hours/week FREE GPU
- Files persist across sessions
- 2x T4 GPUs available
- Pre-installed packages

In [ ]:
# Step 1: Check GPU and Environment
!nvidia-smi

import tensorflow as tf
print(f"\nTensorFlow: {tf.__version__}")
print(f"GPUs: {tf.config.list_physical_devices('GPU')}")
print(f"Working dir: /kaggle/working/")

In [ ]:
# Step 2: Install astronomy packages (Kaggle has most pre-installed)
!pip install -q lightkurve astroquery --upgrade

# Verify
import lightkurve as lk
print(f"Lightkurve: {lk.__version__}")

In [ ]:
# Step 3: Configuration
# Increase these for better accuracy!

NUM_PLANETS = 150        # Exoplanet host stars (increase for better model)
NUM_NON_PLANETS = 150    # Non-planet stars for balance
EPOCHS = 100             # Training epochs
BATCH_SIZE = 64          # Larger batch with GPU
INPUT_SIZE = 1024        # Light curve length
MAX_WORKERS = 12         # Kaggle has good network!

print(f"Training config:")
print(f"  Planets: {NUM_PLANETS}")
print(f"  Non-planets: {NUM_NON_PLANETS}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")

In [ ]:
# Step 4: Fetch exoplanet host list from NASA
import numpy as np
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
import warnings
warnings.filterwarnings('ignore')

print("Querying NASA Exoplanet Archive...")

planets_table = NasaExoplanetArchive.query_criteria(
    table="pscomppars",
    select="hostname,pl_name,disc_facility,pl_orbper,pl_rade",
    where="disc_facility like '%TESS%' or disc_facility like '%Kepler%'"
)

# Get unique host stars
planet_hosts = list(set(planets_table['hostname'].data.tolist()))
np.random.shuffle(planet_hosts)
planet_hosts = planet_hosts[:NUM_PLANETS]

print(f"Found {len(planet_hosts)} exoplanet host stars")
print(f"Examples: {planet_hosts[:5]}")

In [ ]:
# Step 5: Define parallel fetch function
from concurrent.futures import ThreadPoolExecutor, as_completed
import lightkurve as lk

def fetch_lightcurve(args):
    """Fetch and process a single light curve."""
    target, label = args
    try:
        # Search for light curve
        search = lk.search_lightcurve(target, mission=['TESS', 'Kepler'])
        if len(search) == 0:
            return None
        
        # Download and clean
        lc = search[0].download(quality_bitmask='default')
        lc = lc.remove_nans().normalize().remove_outliers(sigma=3)
        
        flux = lc.flux.value
        
        # Resample to fixed size
        if len(flux) < INPUT_SIZE:
            flux = np.pad(flux, (0, INPUT_SIZE - len(flux)), mode='median')
        else:
            start = (len(flux) - INPUT_SIZE) // 2
            flux = flux[start:start + INPUT_SIZE]
        
        return {'flux': flux.astype(np.float32), 'label': label, 'target': target}
    except Exception as e:
        return None

print("Fetch function ready.")

In [ ]:
# Step 6: Fetch planet host light curves (PARALLEL)
from tqdm.notebook import tqdm

print(f"Fetching {NUM_PLANETS} exoplanet hosts with {MAX_WORKERS} workers...")

planet_data = []
tasks = [(host, 1) for host in planet_hosts]

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(fetch_lightcurve, task) for task in tasks]
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Planet hosts"):
        result = future.result()
        if result is not None:
            planet_data.append(result)

print(f"\n✓ Fetched {len(planet_data)} planet host light curves")

In [ ]:
# Step 7: Fetch non-planet stars (negative examples)
print(f"Fetching {NUM_NON_PLANETS} non-planet stars...")

# Use TIC IDs known to NOT have planets
non_planet_tics = [f"TIC {100000000 + i*100}" for i in range(NUM_NON_PLANETS * 5)]

non_planet_data = []
tasks = [(tic, 0) for tic in non_planet_tics]

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(fetch_lightcurve, task) for task in tasks]
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Non-planet stars"):
        if len(non_planet_data) >= NUM_NON_PLANETS:
            break
        result = future.result()
        if result is not None:
            non_planet_data.append(result)

non_planet_data = non_planet_data[:NUM_NON_PLANETS]
print(f"\n✓ Fetched {len(non_planet_data)} non-planet light curves")

In [ ]:
# Step 8: Combine and prepare training data
from sklearn.model_selection import train_test_split

all_data = planet_data + non_planet_data
print(f"Total samples: {len(all_data)}")

X = np.array([d['flux'] for d in all_data])
y = np.array([d['label'] for d in all_data])

# Normalize
X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-8)

# Reshape for CNN: (samples, timesteps, features)
X = X.reshape(-1, INPUT_SIZE, 1).astype(np.float32)

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nData prepared:")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Class balance: {np.bincount(y_train)}")
print(f"  Shape: {X_train.shape}")

In [ ]:
# Step 9: Save training data (Kaggle persists files!)
np.savez('/kaggle/working/larun_training_data.npz',
         X_train=X_train, y_train=y_train,
         X_val=X_val, y_val=y_val,
         targets=[d['target'] for d in all_data])

print("✓ Training data saved to /kaggle/working/larun_training_data.npz")
print("  (This persists across Kaggle sessions!)")

In [ ]:
# Step 10: Build LARUN Model with Multi-GPU support
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Setup multi-GPU strategy (Kaggle has T4 x2)
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

with strategy.scope():
    model = keras.Sequential([
        keras.Input(shape=(INPUT_SIZE, 1)),
        
        # Conv Block 1
        layers.Conv1D(32, 7, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling1D(4),
        layers.Dropout(0.25),
        
        # Conv Block 2
        layers.Conv1D(64, 5, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling1D(4),
        layers.Dropout(0.25),
        
        # Conv Block 3
        layers.Conv1D(128, 3, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.GlobalAveragePooling1D(),
        layers.Dropout(0.5),
        
        # Dense
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(2, activation='softmax')
    ], name='larun_kaggle')
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

model.summary()

In [ ]:
# Step 11: Train with callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        patience=15, 
        restore_best_weights=True,
        monitor='val_accuracy',
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.5, 
        patience=7, 
        min_lr=1e-6,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        '/kaggle/working/larun_best.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1
    )
]

# Adjust batch size for multi-GPU
effective_batch = BATCH_SIZE * strategy.num_replicas_in_sync
print(f"Effective batch size: {effective_batch}")

print("\n" + "="*50)
print("TRAINING STARTED")
print("="*50 + "\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=effective_batch,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Step 12: Evaluate model
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)

print("\n" + "="*50)
print("TRAINING RESULTS")
print("="*50)
print(f"Validation Accuracy: {val_acc*100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Best Accuracy: {max(history.history['val_accuracy'])*100:.2f}%")

In [ ]:
# Step 13: Plot training history
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0].axhline(y=max(history.history['val_accuracy']), color='g', linestyle='--', label=f'Best: {max(history.history["val_accuracy"])*100:.1f}%')
axes[0].set_title('LARUN Model Accuracy', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[1].set_title('LARUN Model Loss', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/training_history.png', dpi=150)
plt.show()

In [ ]:
# Step 14: Export to TFLite for edge deployment
import tensorflow as tf

# Save Keras model
model.save('/kaggle/working/larun_model.h5')

# Standard TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('/kaggle/working/larun_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Quantized TFLite (INT8 for microcontrollers)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant = converter.convert()

with open('/kaggle/working/larun_model_int8.tflite', 'wb') as f:
    f.write(tflite_quant)

print(f"\nModel Sizes:")
print(f"  Keras H5: ~{model.count_params() * 4 / 1024:.1f} KB")
print(f"  TFLite: {len(tflite_model)/1024:.1f} KB")
print(f"  TFLite INT8: {len(tflite_quant)/1024:.1f} KB")

In [ ]:
# Step 15: Save training history and metadata
import json
from datetime import datetime

metadata = {
    'timestamp': datetime.now().isoformat(),
    'platform': 'kaggle',
    'config': {
        'num_planets': NUM_PLANETS,
        'num_non_planets': NUM_NON_PLANETS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'input_size': INPUT_SIZE
    },
    'data': {
        'planet_samples': len(planet_data),
        'non_planet_samples': len(non_planet_data),
        'train_samples': len(X_train),
        'val_samples': len(X_val)
    },
    'results': {
        'val_accuracy': float(val_acc),
        'val_loss': float(val_loss),
        'best_val_accuracy': float(max(history.history['val_accuracy'])),
        'epochs_trained': len(history.history['accuracy'])
    },
    'history': {k: [float(v) for v in vals] for k, vals in history.history.items()}
}

with open('/kaggle/working/training_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✓ Metadata saved")

In [ ]:
# Step 16: Create download package
import shutil
import os

# List all output files
print("Output Files:")
print("="*50)
for f in os.listdir('/kaggle/working/'):
    size = os.path.getsize(f'/kaggle/working/{f}') / 1024
    print(f"  {f}: {size:.1f} KB")

# Create zip
!cd /kaggle/working && zip -r larun_trained_kaggle.zip larun_model.h5 larun_model.tflite larun_model_int8.tflite larun_best.h5 larun_training_data.npz training_metadata.json training_history.png

print("\n" + "="*50)
print("TRAINING COMPLETE!")
print("="*50)
print(f"\nValidation Accuracy: {val_acc*100:.2f}%")
print(f"\nFiles saved to /kaggle/working/")
print("Download 'larun_trained_kaggle.zip' from the Output tab")

## Download Instructions

1. Click **Output** tab on the right sidebar
2. Find `larun_trained_kaggle.zip`
3. Click the **Download** button

Or use the Kaggle API:
```bash
kaggle kernels output <your-username>/<notebook-name>
```

---

**Next Steps:**
1. Copy model files to your LARUN installation
2. Run `/classify` to test the new model
3. Deploy TFLite model to edge devices

---
*Larun Engineering - Astrodata*